## Hacker News data analysis ##

Hacker News is a site started by the startup incubator Y Combinator, where user-submitted stories (known as "posts") are voted and commented upon, similar to reddit. Hacker News is extremely popular in technology and startup circles, and posts that make it to the top of Hacker News' listings can get hundreds of thousands of visitors as a result.

We're specifically interested in posts whose titles begin with either Ask HN or Show HN
We'll compare these two types of posts to determine the following:

- Do Ask HN or Show HN receive more comments on average?
- Do posts created at a certain time receive more comments on average?

First of all, we will read hacker_news.csv file and observe the information available in the dataset.

In [1]:
from csv import reader
opened_file = open('hacker_news.csv')
read_file = reader(opened_file)
data_set = list(read_file)
print(data_set[:5])

[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'], ['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']]


## Extracting headers from a List of lists ##

In [2]:
header = data_set[0]
hn = data_set[1:]
print(header)
print(hn[0:4])

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']
[['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']]


The dataset include information regarding Id, title, url, num_points, num_comments, author, created_at. Below are descriptions of the columns:

**id**: The unique identifier from Hacker News for the post

**title**: The title of the post
**url**: The URL that the posts links to, if it the post has a URL

**num_points**: The number of points the post acquired, calculated as the total number of upvotes minus the total number of downvotes

**num_comments**: The number of comments that were made on the post

**author**: The username of the person who submitted the post

**created_at**: The date and time at which the post was submitted

## Extract HN posts ##

Next step is to extract the posts with HN in it.

In [3]:
ask_posts = []
show_posts = []
other_posts = []
for rows in hn:
    title = rows[1]
    if title.lower().startswith("ask hn"):
        ask_posts.append(rows)
    elif title.lower().startswith("show hn"):
        show_posts.append(rows)
    else:
        other_posts.append(rows)
print(len(ask_posts))
print(len(show_posts))
print(len(other_posts))

1744
1162
17194


In [4]:
print(ask_posts[:5])
print(show_posts[:5])

[['12296411', 'Ask HN: How to improve my personal website?', '', '2', '6', 'ahmedbaracat', '8/16/2016 9:55'], ['10610020', 'Ask HN: Am I the only one outraged by Twitter shutting down share counts?', '', '28', '29', 'tkfx', '11/22/2015 13:43'], ['11610310', 'Ask HN: Aby recent changes to CSS that broke mobile?', '', '1', '1', 'polskibus', '5/2/2016 10:14'], ['12210105', 'Ask HN: Looking for Employee #3 How do I do it?', '', '1', '3', 'sph130', '8/2/2016 14:20'], ['10394168', 'Ask HN: Someone offered to buy my browser extension from me. What now?', '', '28', '17', 'roykolak', '10/15/2015 16:38']]
[['10627194', 'Show HN: Wio Link  ESP8266 Based Web of Things Hardware Development Platform', 'https://iot.seeed.cc', '26', '22', 'kfihihc', '11/25/2015 14:03'], ['10646440', 'Show HN: Something pointless I made', 'http://dn.ht/picklecat/', '747', '102', 'dhotson', '11/29/2015 22:46'], ['11590768', 'Show HN: Shanhu.io, a programming playground powered by e8vm', 'https://shanhu.io', '1', '1', 'h

## Calculating the average number of comments##

In [5]:
total_ask_comments = 0
for rows in ask_posts:
    num_comments = int(rows[4])
    total_ask_comments += num_comments
    
avg_ask_comments = total_ask_comments/len(ask_posts)
print(avg_ask_comments)
    

14.038417431192661


In [6]:
total_show_comments = 0
for rows in show_posts:
    num_comments = int(rows[4])
    total_show_comments += num_comments
    
avg_show_comments = total_show_comments/len(show_posts)
print(avg_show_comments)

10.31669535283993


On average, ask posts receive more comments than show posts. ask posts receive approximately 14 comments while show posts receive 10 comments on average. Since ask posts are more likely to receive comments, we'll focus our remaining analysis just on these posts.

## Finding the amount of Ask Posts and Comments by Hour created ##

Next, we'll determine if ask posts created at a certain time are more likely to attract comments. We'll use the following steps to perform this analysis:

1. Calculate the amount of ask posts created in each hour of the day, along with the number of comments received.
2. Calculate the average number of comments ask posts receive by hour created.

In [7]:
import datetime as dt
result_list = []
for posts in ask_posts:
    result_list.append([posts[6],int(posts[4])])
    
print(result_list[:4])
    

[['8/16/2016 9:55', 6], ['11/22/2015 13:43', 29], ['5/2/2016 10:14', 1], ['8/2/2016 14:20', 3]]


In [9]:
counts_by_hour = {}
comments_by_hour = {}
date_format = "%m/%d/%Y %H:%M"

for row in result_list:
    date = row[0]
    comment = row[1]
    time = dt.datetime.strptime(date, date_format).strftime("%H")
    if time in counts_by_hour:
        comments_by_hour[time] += comment
        counts_by_hour[time] += 1
    else:
        comments_by_hour[time] = comment
        counts_by_hour[time] = 1
        

    

Next, we'll use these two dictionaries to calculate the average number of comments for posts created during each hour of the day.

In [10]:
print(counts_by_hour)
print(comments_by_hour)

{'14': 107, '08': 48, '10': 59, '05': 46, '07': 34, '22': 71, '03': 54, '00': 55, '04': 47, '23': 68, '21': 109, '15': 116, '11': 58, '09': 45, '16': 108, '13': 85, '17': 100, '02': 58, '18': 109, '06': 44, '20': 80, '12': 73, '19': 110, '01': 60}
{'14': 1416, '08': 492, '10': 793, '05': 464, '07': 267, '22': 479, '03': 421, '00': 447, '04': 337, '23': 543, '21': 1745, '15': 4477, '11': 641, '09': 251, '16': 1814, '13': 1253, '17': 1146, '02': 1381, '18': 1439, '06': 397, '20': 1722, '12': 687, '19': 1188, '01': 683}


## Calculating the average number of comments for Ask HN posts by Hour ##

In [11]:
avg_by_hour = []

for time in comments_by_hour:
    avg_by_hour.append([time, comments_by_hour[time]/counts_by_hour[time]])
    
print(avg_by_hour)    

[['14', 13.233644859813085], ['08', 10.25], ['10', 13.440677966101696], ['05', 10.08695652173913], ['07', 7.852941176470588], ['22', 6.746478873239437], ['03', 7.796296296296297], ['00', 8.127272727272727], ['04', 7.170212765957447], ['23', 7.985294117647059], ['21', 16.009174311926607], ['15', 38.5948275862069], ['11', 11.051724137931034], ['09', 5.5777777777777775], ['16', 16.796296296296298], ['13', 14.741176470588234], ['17', 11.46], ['02', 23.810344827586206], ['18', 13.20183486238532], ['06', 9.022727272727273], ['20', 21.525], ['12', 9.41095890410959], ['19', 10.8], ['01', 11.383333333333333]]


## Sorting and printing the values from a List of lists##

In [14]:
swap_avg_by_hour =[]
for rows in avg_by_hour:
    swap_avg_by_hour.append([rows[1],rows[0]])
    
print(swap_avg_by_hour)

[[13.233644859813085, '14'], [10.25, '08'], [13.440677966101696, '10'], [10.08695652173913, '05'], [7.852941176470588, '07'], [6.746478873239437, '22'], [7.796296296296297, '03'], [8.127272727272727, '00'], [7.170212765957447, '04'], [7.985294117647059, '23'], [16.009174311926607, '21'], [38.5948275862069, '15'], [11.051724137931034, '11'], [5.5777777777777775, '09'], [16.796296296296298, '16'], [14.741176470588234, '13'], [11.46, '17'], [23.810344827586206, '02'], [13.20183486238532, '18'], [9.022727272727273, '06'], [21.525, '20'], [9.41095890410959, '12'], [10.8, '19'], [11.383333333333333, '01']]


In [15]:
sorted(swap_avg_by_hour)

[[5.5777777777777775, '09'],
 [6.746478873239437, '22'],
 [7.170212765957447, '04'],
 [7.796296296296297, '03'],
 [7.852941176470588, '07'],
 [7.985294117647059, '23'],
 [8.127272727272727, '00'],
 [9.022727272727273, '06'],
 [9.41095890410959, '12'],
 [10.08695652173913, '05'],
 [10.25, '08'],
 [10.8, '19'],
 [11.051724137931034, '11'],
 [11.383333333333333, '01'],
 [11.46, '17'],
 [13.20183486238532, '18'],
 [13.233644859813085, '14'],
 [13.440677966101696, '10'],
 [14.741176470588234, '13'],
 [16.009174311926607, '21'],
 [16.796296296296298, '16'],
 [21.525, '20'],
 [23.810344827586206, '02'],
 [38.5948275862069, '15']]

In [16]:
sorted_swap = sorted(swap_avg_by_hour, reverse = True)
print(sorted_swap)

[[38.5948275862069, '15'], [23.810344827586206, '02'], [21.525, '20'], [16.796296296296298, '16'], [16.009174311926607, '21'], [14.741176470588234, '13'], [13.440677966101696, '10'], [13.233644859813085, '14'], [13.20183486238532, '18'], [11.46, '17'], [11.383333333333333, '01'], [11.051724137931034, '11'], [10.8, '19'], [10.25, '08'], [10.08695652173913, '05'], [9.41095890410959, '12'], [9.022727272727273, '06'], [8.127272727272727, '00'], [7.985294117647059, '23'], [7.852941176470588, '07'], [7.796296296296297, '03'], [7.170212765957447, '04'], [6.746478873239437, '22'], [5.5777777777777775, '09']]


In [17]:
print("Top 5 Hours for Ask Posts Comments")
print(sorted_swap[:5])

Top 5 Hours for Ask Posts Comments
[[38.5948275862069, '15'], [23.810344827586206, '02'], [21.525, '20'], [16.796296296296298, '16'], [16.009174311926607, '21']]


In [19]:
for avg, hr in sorted_swap[:5]:
    print("{}: {:.2f} average comments per post".format
         (dt.datetime.strptime(hr,"%H").strftime("%H:%M"),avg
         )
         )
     

15:00: 38.59 average comments per post
02:00: 23.81 average comments per post
20:00: 21.52 average comments per post
16:00: 16.80 average comments per post
21:00: 16.01 average comments per post


The hour that receives the most comments per post on average is 15:00, with an average of 38.59 comments per post. There's about a 60% increase in the number of comments between the hours with the highest and second highest average number of comments.

According to the data set documentation, the timezone used is Eastern Time in the US. So, we could also write 15:00 as 3:00 pm est.


## Conclusion##

In this project, we analyzed ask posts and show posts to determine which type of post and time receive the most comments on average. Based on our analysis, to maximize the amount of comments a post receives, we'd recommend the post be categorized as ask post and created between 15:00 and 16:00 (3:00 pm est - 4:00 pm est).

ask posts received more comments on average and ask posts created between 15:00 and 16:00 (3:00 pm est - 4:00 pm est) received the most comments on average.